# Example usage of BatPy's brightway2 export for multiple batteries
`batpy` is able to export a calculated battery from BatPaC as a Life Cycle Inventory Excel file, which can be imported in brightway2. In order to use `batpy` in a project for brightway2 export, follow the example below.

## Import

In [1]:
# included datasets
from batpy import datasets

# batpac battery and tool classes
from batpy.batpac_battery import BatpacBattery
from batpy.batpac_tool import BatpacTool

# brightway2 class
from batpy.brightway import BrightwayConnector

# utility function for combining multiple configuration files
from batpy.utility_functions import combine_configuration

# pathlib for filesystem path handling
from pathlib import Path
from tqdm import tqdm
import shutil


## Paths to Excel files and to TOML configuration

In [ ]:
# Get BatPaC: https://www.anl.gov/partnerships/batpac-battery-manufacturing-cost-estimation
BATPY_BATPAC_EXCEL = Path("./example_data/excel_workbooks/BatPaC.xlsm")

# Brightway workbook is included
BRIGHTWAY2_EXCEL = Path("./example_data/excel_workbooks/BatPaC-Brightway.xlsx")
BRIGHTWAY2_EXCEL_SAVE = Path("./example_data/excel_workbooks")

# brightway2 configuration
BRIGHTWAY_CONFIG = Path(
    "./example_data/conf_brightway/batpy_batpac2brightway.toml"
)

# Battery config
BATPY_BATTERY_CONFIG = Path(
    "./example_data/conf_batpac/batpy_batteries_standard_config.toml"
)


## Get included datasets

Show available dataset of specified version (default latest):

In [3]:
datasets.get_available_batpy_dataset_versions()


[Version('0.3.0'), Version('0.0.0'), Version('0.4.0'), Version('0.1.0')]

In [4]:
datasets.get_available_batpy_datasets()


{'batpy_batteries_config.toml': 'Example configuration for batteries',
 'batpy_batteries_standard_config.toml': 'Standard configuration for batteries',
 'batpy_batpac_config.toml': 'Example configuration for BatPaC tool',
 'batpy_batpac2brightway copy.toml': 'Configuration for brightway2 export',
 'batpy_batpac_summary_of_results.toml': 'Configuration for worksheet Summary of Results in BatPaC Excel',
 'batpy_batpac_chem.toml': 'Configuration for worksheet chem in BatPaC Excel',
 'batpy_batpac_calculation_and_validation_results.toml': 'Configuration for calculation and validation results in BatPaC Excel',
 'batpy_batpac_user_input_cells.toml': 'Configuration for standard user input cells in BatPaC Excel',
 'batpy_batpac_bms.toml': 'Configuration for worksheet BMS in BatPaC Excel',
 'batpy_batpac_battery_design.toml': 'Configuration for worksheet battery design in BatPaC Excel',
 'batpy_batpac2brightway.toml': 'Configuration for brightway2 export'}

Export integrated datasets

In [5]:
datasets.copy_integrated_dataset("batpy_batpac2brightway", BRIGHTWAY_CONFIG)
datasets.copy_integrated_dataset(
    "batpy_batteries_standard_config", BATPY_BATTERY_CONFIG
)


Load multiple included datasets for BatPaC configuration and combine them:

In [6]:
batpy_batpac_dataset = combine_configuration(
    [
        datasets.get_batpy_dataset("batpy_batpac_battery_design"),
        datasets.get_batpy_dataset("batpy_batpac_bms"),
        
    ]
)


## BatPaC tool
### Create BatPaC instance

In [7]:
batpac_excel = BatpacTool(
    BATPY_BATPAC_EXCEL,
    datasets.get_batpy_dataset("batpy_batpac_user_input_cells"),
    None,
    True,
)

## Brightway2 connector
- For each cell chemistry
    - Create batteries
    - Change properties
    - For each battery:
        - Save integrated brightway2 Excel file
        - Create brightway2 instance
        - Load brightway2 configuration and export BatPaC battery data into brightway2 Excel
        - Save Excel file
        - Close Excel file

In [8]:
battery_properties_to_change = {
    ("Dashboard", "Total pack energy, 0-100% SOC (kWh)"): [
        20,
        45,
        68,
        85,
        95,
        100,
        120,
    ]
}


In [9]:
battery_cell_chemistries = [
    "NMC333-G (Power)",
    # "NMC333-G (Energy)",
    "NMC532-G (Power)",
    # "NMC532-G (Energy)",
    "NMC622-G (Power)",
    # "NMC622-G (Energy)",
    "NMC811-G (Power)",
    # "NMC811-G (Energy)",
    "NCA-G (Power)",
    # "NCA-G (Energy)",
    "LFP-G (Power)",
    # "LFP-G (Energy)",
    # "LMO-G (Power)",
    # "LMO-G (Energy)",
    # "LMO-LTO (Power)",
    # "LMO-LTO (Energy)",
    # "50%/50% NMC532/LMO - G",
]

In [10]:
battery_cell_chemistries = [
    "NMC811-G (Power)",
]

In [11]:
def create_batteries(
    cell_chemistry: str, battery_properties: dict
) -> BatpacTool:
    bat1 = BatpacBattery("Battery 1")
    bat2 = BatpacBattery("Battery 2")
    bat3 = BatpacBattery("Battery 3")
    bat4 = BatpacBattery("Battery 4")
    bat5 = BatpacBattery("Battery 5")
    bat6 = BatpacBattery("Battery 6")
    bat7 = BatpacBattery("Battery 7")

    batpac_excel.load_batteries_file(
        BATPY_BATTERY_CONFIG,
        [
            bat1,
            bat2,
            bat3,
            bat4,
            bat5,
            bat6,
            bat7,
        ],
    )

    batpac_excel.set_new_property(
        "Dashboard", "Electrode Couple", cell_chemistry
    )

    for index, battery in enumerate(batpac_excel.batteries):
        for key_1, key_2 in battery_properties:
            battery.set_new_property(
                key_1, key_2, battery_properties[(key_1, key_2)][index]
            )
            
    batpac_excel.calculate()

In [12]:
# def brightway2_export_(battery) -> None:
#     battery_chemistry = batpac_excel.read_value(
#         "Dashboard", "Electrode Couple"
#     )
#     battery_capacity = batpac_excel.read_value(
#         "Dashboard", "Total pack energy, 0-100% SOC (kWh)", battery
#     )
#     activity_name = (
#         f"Battery pack assembly, {battery_chemistry}, {battery_capacity} kWh"
#     )
#     reference_product = (
#         f"Battery system, {battery_chemistry}, {battery_capacity} kWh"
#     )

#     path_to_save = (
#         BRIGHTWAY2_EXCEL_SAVE
#         / f"BatPaC-Brightway_{battery_chemistry}_{battery_capacity}_kWh.xlsx"
#     )
#     datasets.copy_integrated_brightway_workbook(path_to_save)
#     brightway_excel = BrightwayConnector(path_to_save, True)

#     brightway_excel.load_batpac_to_brightway_configuration(BRIGHTWAY_CONFIG)
#     brightway_excel._write_value_direct("Battery Pack", "B5", activity_name)
#     brightway_excel._write_value_direct(
#         "Battery Pack", "B11", reference_product
#     )

#     brightway_excel.export_batpac_battery_to_brightway(
#         batpac=batpac_excel,
#         battery=battery,
#         batpac_config=batpy_batpac_dataset,
#     )

#     brightway_excel.save()
#     brightway_excel.close()

In [13]:
def brightway2_export(battery) -> None:
    battery_chemistry = batpac_excel.read_value(
        "Dashboard", "Electrode Couple"
    )
    battery_capacity = batpac_excel.read_value(
        "Dashboard", "Total pack energy, 0-100% SOC (kWh)", battery
    )
    database_name = (
        f"Battery pack production {battery_chemistry} {battery_capacity} kWh"
    )
    activity_name = (
        f"Battery pack assembly, {battery_chemistry}, {battery_capacity} kWh"
    )
    reference_product = (
        f"Battery system, {battery_chemistry}, {battery_capacity} kWh"
    )

    positive_active_material = batpac_excel.read_value(
        "Chem",
        "Positive Electrode Active Material",
        battery,
        datasets.get_batpy_dataset("batpy_batpac_chem"),
    )

    positive_active_material = (
        f"Positive Active Material {positive_active_material}"
    )

    path_to_copy = BRIGHTWAY2_EXCEL_SAVE / f"BatPaC-Brightway_base.xlsx"

    datasets.copy_integrated_brightway_workbook(path_to_copy)
    brightway_excel = BrightwayConnector(path_to_copy, True)

    brightway_excel.load_batpac_to_brightway_configuration(BRIGHTWAY_CONFIG)
    brightway_excel._write_value_direct("Battery Pack", "B2", database_name)
    brightway_excel._write_value_direct("Battery Pack", "B5", activity_name)
    brightway_excel._write_value_direct(
        "Battery Pack", "B11", reference_product
    )

    brightway_excel._write_value_direct(
        "Pos. Active material", "A14", positive_active_material
    )

    brightway_excel.export_batpac_battery_to_brightway(
        batpac=batpac_excel,
        battery=battery,
        batpac_config=batpy_batpac_dataset,
    )

    path_to_save = (
        BRIGHTWAY2_EXCEL_SAVE
        / f"BatPaC-Brightway_{battery_chemistry}_{battery_capacity}_kWh.xlsx"
    )
    brightway_excel.save(path_to_save)
    brightway_excel.close()

    # recycled
    database_name_recycled = f"Battery pack production recycled {battery_chemistry} {battery_capacity} kWh"
    activity_name_recycled  = (
        f"Battery pack assembly, recycled, {battery_chemistry}, {battery_capacity} kWh"
    )
    reference_product_recycled  = (
        f"Battery system, recycled, {battery_chemistry}, {battery_capacity} kWh"
    )

    path_to_recycled_copy = (
        BRIGHTWAY2_EXCEL_SAVE
        / f"BatPaC-Brightway_recycled_{battery_chemistry}_{battery_capacity}_kWh.xlsx"
    )
    shutil.copy(path_to_save, path_to_recycled_copy)
    brightway_excel = BrightwayConnector(path_to_recycled_copy, True)

    brightway_excel.load_batpac_to_brightway_configuration(BRIGHTWAY_CONFIG)
    brightway_excel._write_value_direct(
        "Battery Pack", "B2", database_name_recycled
    )
    brightway_excel._write_value_direct("Battery Pack", "B5", activity_name_recycled)
    brightway_excel._write_value_direct(
        "Battery Pack", "B11", reference_product_recycled
    )

    brightway_excel.save()
    brightway_excel.close()

In [14]:
for chemistry in battery_cell_chemistries:
    create_batteries(chemistry, battery_properties_to_change)
    for battery in tqdm(
        batpac_excel.batteries,
        "Export each battery as brightway2 inventory",
    ):
        brightway2_export(battery)

Export each battery as brightway2 inventory: 100%|██████████| 7/7 [04:11<00:00, 35.89s/it]


## Close BatPaC file

In [15]:
batpac_excel.close()

True